# Multi-class Text Classification Problem with PySpark and MLlib

On utilise Spark MLlib pour résoudre un problème de classification de texte multiclass peut être extrêmement avantageux pour plusieurs raisons. Voici pourquoi Spark MLlib est une excellente option pour ce type de problème :

### 1. **Traitement Distribué et Scalabilité**
Spark MLlib fait partie de l'écosystème Apache Spark, qui est conçu pour le traitement distribué des données. Cela signifie que vous pouvez traiter des volumes massifs de données textuelles en les répartissant sur plusieurs nœuds de calcul. Cette scalabilité est cruciale pour les problèmes de classification de texte où les ensembles de données peuvent être très volumineux.

### 2. **Intégration Facile avec Spark**
Étant donné que Spark MLlib est intégré dans Apache Spark, il permet une intégration fluide avec les autres composants de Spark, comme Spark SQL et les DataFrames. Cela permet de facilement charger, transformer et préparer les données avant de les utiliser pour l'entraînement du modèle de machine learning.

### 3. **Large Éventail d’Algorithmes de Machine Learning**
Spark MLlib propose une variété d’algorithmes de machine learning adaptés à la classification de texte multiclass, tels que la régression logistique, les machines à vecteurs de support (SVM), et les forêts aléatoires. Cela offre une grande flexibilité dans le choix de l’algorithme le plus adapté à votre problème spécifique.

### 4. **Prétraitement des Données Textuelles**
MLlib inclut des fonctionnalités pour le prétraitement des données textuelles, telles que la tokenization (découpage du texte en mots), la vectorisation (conversion des mots en vecteurs numériques), et l'utilisation du TF-IDF (Term Frequency-Inverse Document Frequency). Ces étapes sont essentielles pour convertir le texte en une forme que les algorithmes de machine learning peuvent comprendre.

### 5. **Pipeline de Machine Learning**
Spark MLlib permet la création de pipelines de machine learning, qui simplifient l'assemblage et l'exécution de différentes étapes du processus de machine learning, telles que le prétraitement des données, la formation du modèle, et l'évaluation. Les pipelines rendent le processus plus modulaire et reproductible.

### 6. **Performance et Efficacité**
Grâce à son modèle de calcul en mémoire, Spark MLlib est très performant pour le traitement de grandes quantités de données. Cela réduit le temps nécessaire pour l'entraînement et l'évaluation des modèles de machine learning, ce qui est particulièrement bénéfique pour les grandes applications de classification de texte.

******************************
## 1. Importation des données 

le problème est de classifier 'La description de crime' en 33 classes

In [1]:
import os
import pandas as pd

Exemple de dataframe avec spark

In [4]:
# Importer findspark et initialiser avec le chemin d'installation de Spark
import findspark
findspark.init('C:\spark')

# Importer PySpark
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder.appName("FirstApp").getOrCreate()

# Exemple de DataFrame avec des données fictives
data = [("James", "Smith", "USA", "CA"), 
        ("Michael", "Rose", "USA", "NY"),
        ("Robert", "Williams", "USA", "CA"),
        ("Maria", "Jones", "USA", "FL")]

columns = ["firstname", "lastname", "country", "state"]

# Créer le DataFrame
df = spark.createDataFrame(data, schema=columns)

# Afficher le DataFrame
df.show()

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|    James|   Smith|    USA|   CA|
|  Michael|    Rose|    USA|   NY|
|   Robert|Williams|    USA|   CA|
|    Maria|   Jones|    USA|   FL|
+---------+--------+-------+-----+



#### Importation de nos données qu'on doit tester

In [7]:
from pyspark.sql import SparkSession

# Arrêtez le SparkContext existant s'il existe
try:
    sc.stop()
except:
    pass

# Créez une nouvelle SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

# Charger le fichier CSV
data = spark.read.format('csv').options(header='true', inferschema='true').load('train.csv')

# Afficher les premières lignes du DataFrame
data.show()

+-------------------+--------------+--------------------+---------+----------+--------------+--------------------+-------------------+------------------+
|              Dates|      Category|            Descript|DayOfWeek|PdDistrict|    Resolution|             Address|                  X|                 Y|
+-------------------+--------------+--------------------+---------+----------+--------------+--------------------+-------------------+------------------+
|2015-05-13 23:53:00|      WARRANTS|      WARRANT ARREST|Wednesday|  NORTHERN|ARREST, BOOKED|  OAK ST / LAGUNA ST|  -122.425891675136|  37.7745985956747|
|2015-05-13 23:53:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday|  NORTHERN|ARREST, BOOKED|  OAK ST / LAGUNA ST|  -122.425891675136|  37.7745985956747|
|2015-05-13 23:33:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday|  NORTHERN|ARREST, BOOKED|VANNESS AV / GREE...|   -122.42436302145|  37.8004143219856|
|2015-05-13 23:30:00| LARCENY/THEFT|GRAND THEFT FROM ...|Wednesday|  NORTHER

******************************
## 2. Preprocessing de nos données

on enlève les colonnes dont on en a pas besoin et on affiche les 5 premières lignes

In [8]:
drop_list = ['Dates', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']
data = data.select([column for column in data.columns if column not in drop_list])

In [9]:
data.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Descript: string (nullable = true)



In [10]:
data.show(5)

+--------------+--------------------+
|      Category|            Descript|
+--------------+--------------------+
|      WARRANTS|      WARRANT ARREST|
|OTHER OFFENSES|TRAFFIC VIOLATION...|
|OTHER OFFENSES|TRAFFIC VIOLATION...|
| LARCENY/THEFT|GRAND THEFT FROM ...|
| LARCENY/THEFT|GRAND THEFT FROM ...|
+--------------+--------------------+
only showing top 5 rows



#### on affiche les 20 classes de crimes les plus fréquents

In [11]:
from pyspark.sql.functions import col
data.groupBy("Category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+------+
|            Category| count|
+--------------------+------+
|       LARCENY/THEFT|174900|
|      OTHER OFFENSES|126182|
|        NON-CRIMINAL| 92304|
|             ASSAULT| 76876|
|       DRUG/NARCOTIC| 53971|
|       VEHICLE THEFT| 53781|
|           VANDALISM| 44725|
|            WARRANTS| 42214|
|            BURGLARY| 36755|
|      SUSPICIOUS OCC| 31414|
|      MISSING PERSON| 25989|
|             ROBBERY| 23000|
|               FRAUD| 16679|
|FORGERY/COUNTERFE...| 10609|
|     SECONDARY CODES|  9985|
|         WEAPON LAWS|  8555|
|        PROSTITUTION|  7484|
|            TRESPASS|  7326|
|     STOLEN PROPERTY|  4540|
|SEX OFFENSES FORC...|  4388|
+--------------------+------+
only showing top 20 rows



#### Le top 20 des descriptions

In [12]:
data.groupBy("Descript") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
|            Descript|count|
+--------------------+-----+
|GRAND THEFT FROM ...|60022|
|       LOST PROPERTY|31729|
|             BATTERY|27441|
|   STOLEN AUTOMOBILE|26897|
|DRIVERS LICENSE, ...|26839|
|      WARRANT ARREST|23754|
|SUSPICIOUS OCCURR...|21891|
|AIDED CASE, MENTA...|21497|
|PETTY THEFT FROM ...|19771|
|MALICIOUS MISCHIE...|17789|
|   TRAFFIC VIOLATION|16471|
|PETTY THEFT OF PR...|16196|
|MALICIOUS MISCHIE...|15957|
|THREATS AGAINST LIFE|14716|
|      FOUND PROPERTY|12146|
|ENROUTE TO OUTSID...|11470|
|GRAND THEFT OF PR...|11010|
|POSSESSION OF NAR...|10050|
|PETTY THEFT FROM ...|10029|
|PETTY THEFT SHOPL...| 9571|
+--------------------+-----+
only showing top 20 rows



******************************
## 3. Création du model de transformation de nos données
### Dans Spark, nous appelons cela la création d'un 'Pipeline de Modèle', et nous accomplissons cela en 5 étapes
1. **regexTokenizer :** Tokenisation (avec expression régulière)
La tokenisation est le processus de découpage du texte en mots (ou tokens). Nous utilisons ici une expression régulière pour identifier et extraire les mots du texte.

2. **stopwordsRemover :** Suppression des mots vides
Les mots vides (stop words) sont des mots courants (comme "et", "le", "à") qui sont souvent supprimés dans les tâches de traitement du langage naturel car ils ne portent pas beaucoup d'information. Cette étape permet de nettoyer le texte en supprimant ces mots non informatifs.

3. **countVectors :** Vecteurs de compte (vecteurs document-terme)
La vectorisation des documents transforme le texte en vecteurs numériques. Chaque document est représenté par un vecteur où chaque dimension correspond à un mot du vocabulaire et la valeur est le nombre d'occurrences de ce mot dans le document. Cela crée ce qu'on appelle des "vecteurs document-terme".

4. **StringIndexer :** Encodage des étiquettes en indices
StringIndexer encode une colonne de chaînes de caractères (étiquettes) en une colonne d'indices d'étiquettes. Les indices sont dans l'intervalle (0, numLabels), ordonnés par fréquences des étiquettes, donc l'étiquette la plus fréquente obtient l'indice 0. Dans notre cas, la colonne d'étiquettes (Category) sera encodée en indices d'étiquettes, de 0 à 32 ; l'étiquette la plus fréquente (LARCENY/THEFT) sera indexée à 0.

5. **Division des données en ensembles d'entraînement et de test pour les rendre prêtes à l'entraînement**
Cette étape consiste à diviser les données disponibles en deux ensembles : un ensemble d'entraînement pour entraîner le modèle, et un ensemble de test pour évaluer la performance du modèle. Cela permet de vérifier que le modèle généralisera bien sur des données qu'il n'a pas vues pendant l'entraînement.

In [13]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, OneHotEncoder, StringIndexer, VectorAssembler, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression

# Clean description using regular-expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="Descript", outputCol="words", pattern="\\W")

# exclue stop words
add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# bag-of-words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

# StringIndexer
label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")

In [14]:
from pyspark.ml import Pipeline

# Put everything in pipeline (We use regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx)
# you can use hasingTF and IDF alternatively than countVectors
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
dataset.show(5)

+--------------+--------------------+--------------------+--------------------+--------------------+-----+
|      Category|            Descript|               words|            filtered|            features|label|
+--------------+--------------------+--------------------+--------------------+--------------------+-----+
|      WARRANTS|      WARRANT ARREST|   [warrant, arrest]|   [warrant, arrest]|(809,[17,32],[1.0...|  7.0|
|OTHER OFFENSES|TRAFFIC VIOLATION...|[traffic, violati...|[traffic, violati...|(809,[11,17,35],[...|  1.0|
|OTHER OFFENSES|TRAFFIC VIOLATION...|[traffic, violati...|[traffic, violati...|(809,[11,17,35],[...|  1.0|
| LARCENY/THEFT|GRAND THEFT FROM ...|[grand, theft, fr...|[grand, theft, fr...|(809,[0,2,3,4,6],...|  0.0|
| LARCENY/THEFT|GRAND THEFT FROM ...|[grand, theft, fr...|[grand, theft, fr...|(809,[0,2,3,4,6],...|  0.0|
+--------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows



In [15]:
# Split Train/Test data
(trainingData, testData) = dataset.randomSplit([0.75, 0.25], seed = 623)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 658091
Test Dataset Count: 219958


******************************
## 4. Entrainement du Modèle et Evaluation :

Notre modèle fera des prédictions et évaluera les scores sur l'ensemble de test.
Ensuite, nous examinerons les 10 meilleures prédictions ayant la plus haute probabilité.<3

In [16]:
# On utilise un modèle de Logistic-Regression 
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Descript","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+-------------+------------------------------+-----+----------+
|                      Descript|     Category|                   probability|label|prediction|
+------------------------------+-------------+------------------------------+-----+----------+
|THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8727757259381659,0.02060...|  0.0|       0.0|
|THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8727757259381659,0.02060...|  0.0|       0.0|
|THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8727757259381659,0.02060...|  0.0|       0.0|
|THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8727757259381659,0.02060...|  0.0|       0.0|
|THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8727757259381659,0.02060...|  0.0|       0.0|
|THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8727757259381659,0.02060...|  0.0|       0.0|
|THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8727757259381659,0.02060...|  0.0|       0.0|
|THEFT, BICYCLE, <$50, SERIA...|LARCENY/THEFT|[0.8

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
print("Test-set Accuracy is : ", evaluator.evaluate(predictions))

Test-set Accuracy is :  0.972984373358745


******************************
## 5. Cross-Validation (Réglage des hyperparamètres)

On va essayer d'améliorer notre modèle et d'ameliorer l'accuracy 

In [18]:
# Same Pipeline step like above
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
(trainingData, testData) = dataset.randomSplit([0.75, 0.25], seed = 623)

# Create LR model
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

In [19]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5])
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) 
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)
cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)

In [20]:
# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
print("Test-set Accuracy is : ", evaluator.evaluate(predictions))

Test-set Accuracy is :  0.9919343876981042


## ON A FINALEMENT OBTENUE 99% D'ACCURACY !


******************************  
 Projet Machine learning en utilisant spark,
 FAIT PAR :                         
* KAMEL Dounia 
* KACHA Selsabile                             
******************************
